# Power System Analysis: Load Flow Techniques

This Notebook is made by:<br> <br> <b> Anmol Tripathi </b>, <br> B.tech Electrical Engineering, <br>Institute of Technology, Nirma University,<br> Ahmedabad, India

## Load Flow: A power system snapshot analysis

...<br>....<br>....

### Load Flow 

<b> Introduction<br>
<del> Ybus Formation </del> <br>
Gauss Siedel Method <br>
DC Load Flow Analysis <br>
Gauss Itterative Method <br>
Newton Raphson Method Polar<br>
Newton Raphson Method Rectangular<br>
Decoupled Method <br>
Fast Decoupled Method <br></b>

### Economic Load Dispatch

<b> Newton Raphson Method<br>
Advanced Methodologies<br></b>

# Contents

* [Introduction](#Introduction) <br>

* [Handling data](#Handling-data) <br>
> [Line and Bus Data](#Line-and-bus-data) <br>
>> [Entering the Line Data](#Entering-the-line-data) <br>
>> [Writing the line data to a CSV file](#Writing-the-line-data-to-a-CSV-file) <br>
>> [Reading the line data from a CSV file](#Reading-the-line-data-from-a-CSV-file) <br>

* [Y<sub>bus</sub> and Z<sub>bus</sub> Formation](#Ybus-formation) <br>
* [DC Loadflow Analysis](#DC-Load-Flow-Analysis) <br>

...<br>....<br>....

# Introduction

Assumptions made while performing load flow:
* Line Resistance being small, are neglected, i.e. Active power loss in the system is zero
* Considering stability, hence 
$\sin(\delta_i - \delta_k) \approx (\delta_i - \delta_k)$ (As, for small values of $\theta$, $\sin\theta \approx \theta$)

<br>...<br>....<br>....

In [47]:
import numpy as np
import os
import cmath
import csv
import pandas as pd
import queue
from tkinter import *
import tkinter.messagebox

# Handling data

### Checking Matrix a is symmetric?

In [7]:
def check_symmetric(a, tol=1e-8):
    return np.all(np.abs(a-a.T) < tol)

### Checking a matrix is invertible? 
 

In [34]:
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

<br>...<br>....<br>.... About line and Bus data

## Line data

### Entering the line data

<br>...<br>....<br>.... <br> WORKING ON GUI Based

In [13]:
def enter_line_data():
    nb = int(input("Enter the number of Buses: ")) 
    nl = int(input("Enter the number of Lines: "))
    from_bus = []
    to_bus = []
    R = []
    X = []
    Z = []
    Y = []
    B_half = []
    turns_ratio = []
    for i in range(0,nl):
        print("\n\n Entry Number: ", i+1)
        x = float(input("\n Enter the from bus: "))
        if(x<=nb):
            from_bus.append(x)
        x = float(input("\n Enter the to bus: "))
        if(x<=nb):
            to_bus.append(x)
        x = float(input("\n Enter the Line Resistance: "))
        R.append(x)
        x = float(input("\n Enter the Line Reactance: "))
        X.append(x)
        x = float(input("\n Enter the Line B/2: i"))
        x = complex(0,x)
        B_half.append(x)
        x = complex(R[i],X[i])
        g = complex("{0:.2f}".format(x))
        Z.append(g)
        print("\n Line Impedence = ", Z[i])
        Y.append(1/Z[i])
        print("\n Line Admittance = ", Y[i])
        x = float(input("\n Enter the turns ratio: "))
        turns_ratio.append(x)
    data_matrix = [(from_bus[i], to_bus[i], R[i], X[i], B_half[i], Y[i], turns_ratio[i]) for i in range(0, nl)] 
    data_matrix = pd.DataFrame(data_matrix, columns=['From_Bus','To_Bus','R','X','B_half','Y','Turns_Ratio'])
    return data_matrix

In [12]:
data_matrix = enter_line_data()

Enter the number of Buses: 6
Enter the number of Lines: 11


 Entry Number:  1

 Enter the from bus: 1

 Enter the to bus: 2

 Enter the Line Resistance: 0.10

 Enter the Line Reactance: 0.20

 Enter the Line B/2: i0.02

 Line Impedence =  (0.1+0.2j)

 Line Admittance =  (2-4j)

 Enter the turns ratio: 1


 Entry Number:  2

 Enter the from bus: 1

 Enter the to bus: 4

 Enter the Line Resistance: 0.05

 Enter the Line Reactance: 0.20

 Enter the Line B/2: i0.02

 Line Impedence =  (0.05+0.2j)

 Line Admittance =  (1.176470588235294-4.705882352941176j)

 Enter the turns ratio: 1


 Entry Number:  3

 Enter the from bus: 1

 Enter the to bus: 5

 Enter the Line Resistance: 0.08

 Enter the Line Reactance: 0.30

 Enter the Line B/2: i0.03

 Line Impedence =  (0.08+0.3j)

 Line Admittance =  (0.8298755186721992-3.1120331950207474j)

 Enter the turns ratio: 1


 Entry Number:  4

 Enter the from bus: 2

 Enter the to bus: 3

 Enter the Line Resistance: 0.05

 Enter the Line Reactance: 0.25

In [13]:
data_matrix

,From_Bus,To_Bus,R,X,B_half,Y,Turns_Ratio
0,1.0,2.0,0.10,0.20,0.02j,(2-4j),1.0
1,1.0,4.0,0.05,0.20,0.02j,(1.176470588235294-4.705882352941176j),1.0
2,1.0,5.0,0.08,0.30,0.03j,(0.8298755186721992-3.1120331950207474j),1.0
3,2.0,3.0,0.05,0.25,0.03j,(0.7692307692307693-3.846153846153846j),1.0
4,2.0,4.0,0.05,0.10,0.01j,(4-8j),1.0
5,2.0,5.0,0.10,0.30,0.02j,(1.0000000000000002-3j),1.0
6,2.0,6.0,0.07,0.20,0.025j,(1.55902004454343-4.4543429844097995j),1.0
7,3.0,5.0,0.12,0.26,0.025j,(1.4634146341463414-3.1707317073170733j),1.0
8,3.0,6.0,0.02,0.10,0.01j,(1.9230769230769227-9.615384615384615j),1.0
9,4.0,5.0,0.20,0.40,0.04j,(1-2j),1.0


### Writing the line data to a CSV file

<br>...<br>....<br>....

In [8]:
def line_write_to_csv(data_matrix):
    dir_path = os.getcwd()
    filename = input("Enter the Filename: ")
    filename = filename + '.csv'
    export_csv = data_matrix.to_csv (os.path.join(dir_path,filename), index = None, header=True)

In [17]:
line_write_to_csv(data_matrix)

Enter the Filename: line_data


### Reading the line data from a CSV file

<br>...<br>....<br>....

In [3]:
# Bus Data Example:
#           |  From |  To   |   R   |   X   |   B/2 |  Turns Ratio |
#           |  Bus  | Bus   |       |       |       |              |
# linedata = [ 1      2       0.10    0.20     0.02         1;
#              1      4       0.05    0.20     0.02         1;
#              1      5       0.08    0.30     0.03         1;
#              2      3       0.05    0.25     0.03         1;
#              2      4       0.05    0.10     0.01         1;
#              2      5       0.10    0.30     0.02         1;
#              2      6       0.07    0.20     0.025        1;
#              3      5       0.12    0.26     0.025        1;
#              3      6       0.02    0.10     0.01         1;
#              4      5       0.20    0.40     0.04         1;
#              5      6       0.10    0.30     0.03         1;];

def line_read_from_csv():
    print("\n\n Please make sure that the CSV has data in following format: \n\n\n From Bus\tTo Bus\t R\tX\tZ\tG\tB\tY\tTurns Ratio\n \n\n Note: The csv file should be in the same directory")
    filename = input("\n Enter the CSV Filename:  ")
    filename = filename + '.csv' 
    dir_path = os.getcwd()
    csv_path = os.path.join(dir_path, filename)
    data = pd.read_csv(csv_path)
    data.head()
    return(data)

In [48]:
data_matrix = line_read_from_csv()
data_matrix



 Please make sure that the CSV has data in following format: 


 From Bus	To Bus	 R	X	Z	G	B	Y	Turns Ratio
 

 Note: The csv file should be in the same directory

 Enter the CSV Filename:  line_data


,From_Bus,To_Bus,R,X,B_half,Y,Turns_Ratio
0,1.0,2.0,0.10,0.20,0.02j,(2-4j),1.0
1,1.0,4.0,0.05,0.20,0.02j,(1.176470588235294-4.705882352941176j),1.0
2,1.0,5.0,0.08,0.30,0.03j,(0.8298755186721992-3.1120331950207474j),1.0
3,2.0,3.0,0.05,0.25,0.03j,(0.7692307692307693-3.846153846153846j),1.0
4,2.0,4.0,0.05,0.10,0.01j,(4-8j),1.0
5,2.0,5.0,0.10,0.30,0.02j,(1.0000000000000002-3j),1.0
6,2.0,6.0,0.07,0.20,0.025j,(1.55902004454343-4.4543429844097995j),1.0
7,3.0,5.0,0.12,0.26,0.025j,(1.4634146341463414-3.1707317073170733j),1.0
8,3.0,6.0,0.02,0.10,0.01j,(1.9230769230769227-9.615384615384615j),1.0
9,4.0,5.0,0.20,0.40,0.04j,(1-2j),1.0


## Bus Data

<br>...<br>....<br>....

### Reading the bus data from a CSV file

<br>...<br>....<br>....

In [49]:
#BUS DATA EXAMPLE:

#           |Bus | Type | Vsp | theta | PGi | QGi | PLi | QLi | Qmin | Qmax |
# busdata = [ 1   Slack  1.05     0     0.0    0    0      0      0      0;
#             2    PV    1.05     0     0.5    0    0      0    -0.5   1.0;
#             3    PV    1.07     0     0.6    0    0      0    -0.5   1.5;
#             4    PQ    1.0      0     0.0    0    0.7    0.7     0     0;
#             5    PQ    1.0      0     0.0    0    0.7    0.7     0     0;
#             6    PQ    1.0      0     0.0    0    0.7    0.7     0     0 ];


def bus_read_from_csv():
    print("\n\n Please make sure that the CSV has data in following format: \n\n\n Bus Number\tBus Type\tVsp\ttheta\tPgi\tQgi\tPdi\tQdi\tQmin\tQmax \n\n Note: The csv file should be in the same directory")
    filename = input("\n Enter the CSV Filename:  ")
    filename = filename + '.csv'
    dir_path = os.getcwd()
    csv_path = os.path.join(dir_path, filename)
    data = pd.read_csv(csv_path)
    data.head()
    return(data)

In [51]:
bus_data_matrix = bus_read_from_csv()
bus_data_matrix



 Please make sure that the CSV has data in following format: 


 Bus Number	Bus Type	Vsp	theta	Pgi	Qgi	Pdi	Qdi	Qmin	Qmax 

 Note: The csv file should be in the same directory

 Enter the CSV Filename:  bus_data


,Bus_no,Bus_type,Vsp,theta,Pgi,Qgi,Pdi,Qdi,Qmin,Qmax
0,1,Slack,1.05,0,0.0,0,0.0,0.0,0.0,0.0
1,2,PV,1.05,0,0.5,0,0.0,0.0,-0.5,1.0
2,3,PV,1.07,0,0.6,0,0.0,0.0,-0.5,1.5
3,4,PQ,1.00,0,0.0,0,0.7,0.7,0.0,0.0
4,5,PQ,1.00,0,0.0,0,0.7,0.7,0.0,0.0
5,6,PQ,1.00,0,0.0,0,0.7,0.7,0.0,0.0


# Bus Incidence Matrix 

...<br>
...<br>
...

In [9]:
def make_bus_incidence_matrix(datamatrix):
    linedata = datamatrix
    frombus = linedata.iloc[:,0].astype(int) #Because float isn't itteratable
    tobus = linedata.iloc[:,1].astype(int)
    no_bus = max(max(frombus),max(tobus)) #To find number of buses 
    bus_incidence = np.zeros((no_bus,no_bus)).astype(int)
    for i in range(0,no_bus):
        bus_incidence[frombus[i]][tobus[i]] = -1
        bus_incidence[tobus[i]][frombus[i]] = -1
    np.fill_diagonal(bus_incidence,1)
    bus_incidence = np.delete(bus_incidence,0,0)
    bus_incidence = np.delete(bus_incidence,0,1)
    return bus_incidence

In [10]:
bus_incidence =  make_bus_incidence_matrix(data_matrix)
print(bus_incidence)

[[ 1 -1  0 -1 -1]
 [-1  1 -1 -1 -1]
 [ 0 -1  1  0  0]
 [-1 -1  0  1  0]
 [-1 -1  0  0  1]]


In [24]:
check_symmetric(bus_incidence)

True

# Y<sub>bus</sub> and Z<sub>bus</sub> Formation

<br>...<br>....<br>....<br>...<br>....<br>....

### Y<sub>bus</sub> Formation

In [52]:
def make_ybus(datamatrix,bus_count_required=0):
    linedata = datamatrix
    frombus = linedata.iloc[:,0].astype(int) #Because float isn't itteratable
    tobus = linedata.iloc[:,1].astype(int)
    no_bus = max(max(frombus),max(tobus)) #To find number of buses 
    no_line = frombus.size
    y_bus = np.zeros((no_bus,no_bus)).astype(complex)
    for i in range(0,no_line):
        y_bus[frombus[i]-1,tobus[i]-1] = -(complex(linedata.iloc[i,5]))
        y_bus[tobus[i]-1,frombus[i]-1] = -(complex(linedata.iloc[i,5]))
        y_bus[frombus[i]-1,frombus[i]-1] = y_bus[frombus[i]-1,frombus[i]-1] + (complex(linedata.iloc[i,5]))
        y_bus[tobus[i]-1,tobus[i]-1] = y_bus[tobus[i]-1,tobus[i]-1] + (complex(linedata.iloc[i,5]))
    print("Ybus Formed")
    if(bus_count_required==0):
        return y_bus
    elif(bus_count_required==1):
        return y_bus,no_bus

In [84]:
ybus = make_ybus(data_matrix,0)

Ybus Formed


In [85]:
ybus

array([[ 4.00634611-11.81791555j, -2.         +4.j        ,
         0.         +0.j        , -1.17647059 +4.70588235j,
        -0.82987552 +3.1120332j ,  0.         +0.j        ],
       [-2.         +4.j        ,  9.32825081-23.30049683j,
        -0.76923077 +3.84615385j, -4.         +8.j        ,
        -1.         +3.j        , -1.55902004 +4.45434298j],
       [ 0.         +0.j        , -0.76923077 +3.84615385j,
         4.15572233-16.63227017j,  0.         +0.j        ,
        -1.46341463 +3.17073171j, -1.92307692 +9.61538462j],
       [-1.17647059 +4.70588235j, -4.         +8.j        ,
         0.         +0.j        ,  6.17647059-14.70588235j,
        -1.         +2.j        ,  0.         +0.j        ],
       [-0.82987552 +3.1120332j , -1.         +3.j        ,
        -1.46341463 +3.17073171j, -1.         +2.j        ,
         5.29329015-14.2827649j , -1.         +3.j        ],
       [ 0.         +0.j        , -1.55902004 +4.45434298j,
        -1.92307692 +9.61538462j,  

In [79]:
check_symmetric(ybus)

True

### Saving Y<sub>bus</sub> as CSV

In [80]:
def write_ybus_to_csv(ybus):
    y_bus_df = pd.DataFrame(ybus)
    dir_path = os.getcwd()
    filename = input("Enter the Filename: ")
    filename = filename + '.csv'
    export_csv = y_bus_df.to_csv (os.path.join(dir_path,filename), index = None, header=True)
    print("File Successfully Created")
    return y_bus_df

In [81]:
ybus_df = write_ybus_to_csv(ybus)
ybus_df

Enter the Filename: y_bus
File Successfully Created


### Z<sub>bus</sub> Formation

In [18]:
def make_zbus(datamatrix):
    ybus = make_ybus(datamatrix,0)
    ybus_shape  = ybus.shape[0]
    zbus = np.zeros(ybus_shape)
    try:
        zbus = np.linalg.inv(ybus)
        return zbus
    except:
        print("Given matrix is SINGULAR")
        try:
            print("Trying Pseudoinverse")
            zbus = np.linalg.pinv(ybus)
            return zbus
        except:
            print("Inverse Not Possible!")

In [19]:
zbus = make_zbus(data_matrix)
zbus

Ybus Formed


array([[-9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j],
       [-9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j],
       [-9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j],
       [-9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j],
       [-9.25397184e+13+2.46772582e+14j, -9.25397184e+13+2.46772582e+14j,
        -9.25397184e+13+2.46772582

In [20]:
def write_zbus_to_csv(zbus):
    z_bus_df = pd.DataFrame(zbus)
    dir_path = os.getcwd()
    filename = input("Enter the Filename: ")
    filename = filename + '.csv'
    export_csv = z_bus_df.to_csv (os.path.join(dir_path,filename), index = None, header=True)
    print("File Successfully Created")
    return z_bus_df

In [21]:
zbus_df = write_zbus_to_csv(zbus)
zbus_df

Enter the Filename: z_bus
File Successfully Created


# Gauss Siedel Method

Gauss Siedel is an itterative algorithm for solving set of non-linear algebraic equations

As we know, <br>
<img src="images/Vi.jpg">
<br><br><br><br>
<img src="images/Qi.jpg">
<br><br><br><br>
<img src="images/Gauss-Seidel-Method-Flowchart.jpg">


In [77]:
# def Gauss_Siedel_Load_Flow(linedata, busdata, ittr_limit = 15, tolerance = 0.0001):
    
#     #`````````````Initializations````````````````#
#     ybus,no_bus = make_ybus(linedata,1)
#     Vinitial = cmath.polar(complex(1,0))
#     ittr = 0
#     buscount = 2
#     max_change_in_V = 0
    
# #    Vi = cmath.rect(complex(busdata['V'],busdata['delta']))
#     Vi = [Vinitial for i in range(0,no_bus)] #For all PV/PQ bus, V = 1L0
#     Qi = complex((busdata['Pg'] - busdata['Pd']),(busdata['Qg'] - busdata['Qd']))
#     bus_type = busdata['Bus_type']
#     #````````````````````````````````````````````#
# #     A = complex(busdata['P'],busdata['Q'])
# #     B = linedata['Y']/(np.diagonal['Y'])
#     while((max_change_in_V > tolerance) and (ittr < ittr_limit)):
#         buscount = 2
#         change_in_V = 0
#         while(buscount<=no_bus):
#             if(busdata[i]['Bus_type'] == 'PV'):
#                 #````````````Computation of Qi ````````````#
#                 for k in range(0,i):
#                     busdata[i]['Q'] = (-imag(complex(np.conj(Vi.get()) * ybus[buscount][k] * Vi.get() )))
                           
#         ittr = ittr + 1

        

In [64]:
def Gauss_Siedel_Load_Flow(linedata, busdata, ittr_limit = 15, tolerance = 0.0001):
    ybus, no_bus = make_ybus(linedata, 1)
    bus = busdata['Bus_no'].astype(int)
    bustype = busdata['Bus_type']
    V = busdata['Vsp'].astype(complex)
    theta = busdata['theta'].astype(float)
    GenMW = busdata['Pgi'].astype(float)
    GenMVAR = busdata['Qgi'].astype(float)
    LoadMW = busdata['Pdi'].astype(float)
    LoadMVAR = busdata['Qdi'].astype(float)
    Qmin = busdata['Qmin'].astype(float)
    Qmax = busdata['Qmax'].astype(float)
    P = GenMW - LoadMW
    Q = GenMVAR - LoadMVAR
    Vittr = V
    V_delta = 1
    ittr = 1
    while(V_delta>tolerance):
        for i in range(1,no_bus):
            sumyv = 0
            for k in range(0,no_bus):
                if(i!=k):
                    sumyv = sumyv + ybus[i][k] * V[k]
            if(bustype[i]=='PV'):
                Q[i] = -(np.conj(V[i]) * (sumyv + ybus[i][i]*V[i]) ).imag
                if(Q[i]>Qmax[i] or Q[i]<Qmin[i]):    
                    if(Q[i] > Qmax[i]):
                        Q[i] = Qmin[i]
                    else:
                        Q[i] = Qmax[i]
                    bustype[i] = 'PQ'
            V[i] = (1/ybus[i,i])*(np.conj(complex(P[i],Q[i]))/sumyv)
            if(bustype[i] =='PV'):
                V[i] = cmath.rect(abs(Vittr[i]), cmath.phase(V[i]))
        ittr = ittr+1
        V_delta = max(abs(abs(V)-abs(Vittr)))
        Vittr = V
    print("Itteration: = ",ittr)
    print("Voltage: = ",V)
    Vmag = abs(V)
    Ang = []
    for i in range(0,len(V)):
        Ang.append(180/cmath.pi * cmath.phase(V[i]))
    print("Vmag: = " ,Vmag)
    print("Angle: = ",Ang)
    Gauss_Siedel = [(Vmag[i], Ang[i], V[i], i) for i in range(0, len(V))] 
    Gauss_Siedel = pd.DataFrame(Gauss_Siedel, columns=['V_magnitude','Angle','Voltage','itterations'])
    return Gauss_Siedel

In [65]:
G = Gauss_Siedel_Load_Flow(data_matrix,bus_data_matrix)
G

Ybus Formed
Itteration: =  2
Voltage: =  0                                           (1.05+0j)
1    (-0.00021987351069373528-0.0013606781191826233j)
2      (0.0023180118005767863-0.0012764649233992641j)
3      (-0.0006624688598894915+0.008485363937866832j)
4      (-0.0013727606942280143+0.009835447324397966j)
5            (1.0013105975688128+2.8918720761274503j)
Name: Vsp, dtype: complex128
Vmag: =  0    1.050000
1    0.001378
2    0.002646
3    0.008511
4    0.009931
5    3.060318
Name: Vsp, dtype: float64
Angle: =  [0.0, -99.1791440833808, -28.84035984706797, 94.46413761698841, 97.94560144933476, 70.90159035356298]


,V_magnitude,Angle,Voltage,itterations
0,1.050000,0.000000,(1.05+0j),0
1,0.001378,-99.179144,(-0.00021987351069373528-0.0013606781191826233j),1
2,0.002646,-28.840360,(0.0023180118005767863-0.0012764649233992641j),2
3,0.008511,94.464138,(-0.0006624688598894915+0.008485363937866832j),3
4,0.009931,97.945601,(-0.0013727606942280143+0.009835447324397966j),4
5,3.060318,70.901590,(1.0013105975688128+2.8918720761274503j),5


### Save Gauss Siedel as CSV

In [68]:
def write_gauss_siedel_to_csv(gs):
    dir_path = os.getcwd()
    filename = input("Enter the Filename: ")
    filename = filename + '.csv'
    export_csv = gs.to_csv (os.path.join(dir_path,filename), index = None, header=True)
    print("File Successfully Created")

In [69]:
write_gauss_siedel_to_csv(G)

Enter the Filename: gauss_siedel_results
File Successfully Created


# DC Loadflow

...<br>
...<br>
...<br>

In [41]:
def dcloadflow():
    datamatrix = line_read_from_csv()
    #datamatrix = datamatrix.drop(['R','B_half','Turns_Ratio'],axis = 1)
    X = datamatrix['X'].astype(float)
    iX = []
    for i in range(0,len(X)):
        iX.append(complex(0,X[i]))
    datamatrix['Y'] = iX
    y_bus_dclf = make_ybus(datamatrix)
    x_bus_dclf = np.linalg.pinv(y_bus_dclf)
    print(x_bus_dclf)
    #print(is_invertible(y_bus_dclf))
    return y_bus_dclf

In [42]:
data = dcloadflow()
data



 Please make sure that the CSV has data in following format: 


 From Bus	To Bus	 R	X	Z	G	B	Y	Turns Ratio
 

 Note: The csv file should be in the same directory

 Enter the CSV Filename:  line_data
Ybus Formed
[[-2.93173721e-17-1.11472658j -1.69625701e-17+0.15846977j
   1.24524382e-16+0.44485876j -6.69021536e-17-0.01240411j
   6.31796397e-17+0.07937199j  7.65007282e-17+0.44443016j]
 [-1.56235970e-17+0.15846977j -8.83081783e-17-0.67581063j
  -4.83947367e-17+0.05803868j  1.26291711e-16+0.24483996j
  -1.30882232e-16+0.10152103j  1.03814704e-16+0.11294119j]
 [ 5.56463871e-17+0.44485876j  9.65347126e-17+0.05803868j
  -9.30658505e-17-1.25627579j  7.98369810e-17+0.45408061j
  -1.87722418e-17+0.14103535j -1.08398699e-16+0.15826238j]
 [-6.73559835e-17-0.01240411j -1.23406927e-16+0.24483996j
   2.34572197e-18+0.45408061j  6.93660307e-17-1.14871055j
   4.84582646e-18+0.01808193j -2.97511737e-17+0.44411216j]
 [-2.58065348e-17+0.07937199j -6.87739873e-17+0.10152103j
   1.10128074e-17+0.14103535j 

array([[ 0.+0.7j , -0.-0.2j ,  0.+0.j  , -0.-0.2j , -0.-0.3j ,  0.+0.j  ],
       [-0.-0.2j ,  0.+1.05j, -0.-0.25j, -0.-0.1j , -0.-0.3j , -0.-0.2j ],
       [ 0.+0.j  , -0.-0.25j,  0.+0.61j,  0.+0.j  , -0.-0.26j, -0.-0.1j ],
       [-0.-0.2j , -0.-0.1j ,  0.+0.j  ,  0.+0.7j , -0.-0.4j ,  0.+0.j  ],
       [-0.-0.3j , -0.-0.3j , -0.-0.26j, -0.-0.4j ,  0.+1.56j, -0.-0.3j ],
       [ 0.+0.j  , -0.-0.2j , -0.-0.1j ,  0.+0.j  , -0.-0.3j ,  0.+0.6j ]])